In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
import glob
from sklearn.linear_model import LinearRegression
import numpy as np

#folder_path should be set to the directory containing the Excel files for each individual ring-on-ring
folder_path = r'C:\Users\ragha\OneDrive\Documenten\Bsc 3\Bep\RTest'
all_files = sorted(glob.glob(os.path.join(folder_path, '*.xlsx')))

results = []
LOAD_THRESHOLD = 1.0
plots_per_figure = 30

pdf_deflection = PdfPages(os.path.join(folder_path, 'deflection_subplots.pdf'))
pdf_load = PdfPages(os.path.join(folder_path, 'load_subplots.pdf'))

subplot_idx = 0
fig_def, axes_def = None, None
fig_load, axes_load = None, None

for i, file in enumerate(all_files):
    try:
        df = pd.read_excel(file, decimal=',')
        print(f"🔍 Processing: {os.path.basename(file)}")

        col_load = 'Load'
        col_time = 'Time'
        col_deflection = 'Avg_S1_S2' if 'Avg_S1_S2' in df.columns else 'S1'

        df_test = df[df[col_load] > LOAD_THRESHOLD].copy()
        if df_test.empty:
            print(f"⚠️ No data above load threshold in {os.path.basename(file)}")
            continue

        # Linear regression on actual test time
        X = df_test[[col_time]].values
        y = df_test[col_deflection].values
        model = LinearRegression().fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_
        y_pred = model.predict(X)

        # Extrapolate to find t0 (when trendline hits deflection = 0)
        t0 = -intercept / slope if slope != 0 else np.nan

        idx_max = df_test[col_load].idxmax()
        failure_time = df_test.loc[idx_max, col_time]
        failure_deflection = df_test.loc[idx_max, col_deflection]
        max_load = df_test.loc[idx_max, col_load]
        temp = df_test.loc[idx_max, 'T'] if 'T' in df.columns else None
        hum = df_test.loc[idx_max, 'H'] if 'H' in df.columns else None

        relative_failure_time = failure_time - t0 if not np.isnan(t0) else np.nan

        results.append({
            'filename': os.path.basename(file),
            'max_load_N': max_load * 1000,
            'failure_deflection': failure_deflection,
            'failure_time_s': relative_failure_time,
            'failure_rate_slope': slope,
            'temperature_C': temp,
            'humidity_%': hum
        })

        if i % plots_per_figure == 0:
            if fig_def:
                fig_def.tight_layout(pad=3.0)
                pdf_deflection.savefig(fig_def)
                plt.close(fig_def)
            if fig_load:
                fig_load.tight_layout(pad=3.0)
                pdf_load.savefig(fig_load)
                plt.close(fig_load)
            fig_def, axes_def = plt.subplots(6, 5, figsize=(20, 24))
            fig_load, axes_load = plt.subplots(6, 5, figsize=(20, 24))
            axes_def = axes_def.flatten()
            axes_load = axes_load.flatten()
            subplot_idx = 0

        # DEFLECTION PLOT
        ax_def = axes_def[subplot_idx]
        ax_def.plot(df_test[col_time], df_test[col_deflection], label='Deflection', color='blue')

        # Trendline from extrapolated t0 to last point
        t_last = df_test[col_time].iloc[-1]
        x_trend = np.array([t0, t_last])
        y_trend = slope * x_trend + intercept
        ax_def.plot(x_trend, y_trend, linestyle='--', color='yellow', label='Trendline')

        # Equation of trendline at the end
        x_label = x_trend[1]
        y_label = y_trend[1]
        trend_eq = f"y = {slope:.4f}x + {intercept:.2f}"
        ax_def.text(x_label, y_label, trend_eq,
                    fontsize=6, verticalalignment='bottom', horizontalalignment='left',
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="white", edgecolor="gray"))

        ax_def.scatter(failure_time, failure_deflection, color='red', label='Failure point')
        ax_def.axvline(failure_time, color='red', linestyle=':')

        if not np.isnan(t0) and t0 < df_test[col_time].max():
            ax_def.axvline(t0, color='green', linestyle='--', label='t0 (start loading)')
            ax_def.scatter(t0, 0, color='green', marker='x', s=50)

        ax_def.set_title(os.path.basename(file), fontsize=8)
        ax_def.set_xlabel('Time (s)')
        ax_def.set_ylabel('Deflection (mm)')
        ax_def.legend(fontsize=6, loc='upper left')
        ax_def.grid(True)

        # LOAD PLOT
        ax_load = axes_load[subplot_idx]
        ax_load.plot(df_test[col_time], df_test[col_load], label='Load (kN)', color='purple')
        ax_load.scatter(failure_time, max_load, color='red', label='Failure point')
        ax_load.axvline(failure_time, color='red', linestyle=':')
        if not np.isnan(t0):
            ax_load.axvline(t0, color='green', linestyle='--', label='t0 (start loading)')
            ax_load.scatter(t0, 0, color='green', marker='x', s=50)

        ax_load.set_title(os.path.basename(file), fontsize=8)
        ax_load.set_xlabel('Time (s)')
        ax_load.set_ylabel('Load (kN)')
        ax_load.legend(fontsize=6, loc='upper left')
        ax_load.grid(True)

        subplot_idx += 1

    except Exception as e:
        print(f"❌ Error processing {os.path.basename(file)}: {e}")

# Save last figures
if fig_def:
    fig_def.tight_layout(pad=3.0)
    pdf_deflection.savefig(fig_def)
    plt.close(fig_def)
if fig_load:
    fig_load.tight_layout(pad=3.0)
    pdf_load.savefig(fig_load)
    plt.close(fig_load)

pdf_deflection.close()
pdf_load.close()

# Save results
df_results = pd.DataFrame(results)
column_order = [
    'filename',
    'max_load_N',
    'failure_deflection',
    'failure_time_s',
    'failure_rate_slope',
    'temperature_C',
    'humidity_%'
]
df_results = df_results[column_order]
output_path = os.path.join(folder_path, 'failure_analysisfinal1.xlsx')
df_results.to_excel(output_path, index=False)

print(f"✅ Analysis complete.\n📄 Results: {output_path}\n📊 PDF plots saved.")

🔍 Processing: AR-Air 1.xlsx
🔍 Processing: AR-Air 10.xlsx
🔍 Processing: AR-Air 11.xlsx
🔍 Processing: AR-Air 12.xlsx
🔍 Processing: AR-Air 13.xlsx
🔍 Processing: AR-Air 14.xlsx
🔍 Processing: AR-Air 15.xlsx
🔍 Processing: AR-Air 16.xlsx
🔍 Processing: AR-Air 17.xlsx
🔍 Processing: AR-Air 18.xlsx
🔍 Processing: AR-Air 19.xlsx
🔍 Processing: AR-Air 2.xlsx
🔍 Processing: AR-Air 20.xlsx
🔍 Processing: AR-Air 21.xlsx
🔍 Processing: AR-Air 22.xlsx
🔍 Processing: AR-Air 23.xlsx
🔍 Processing: AR-Air 24.xlsx
🔍 Processing: AR-Air 25.xlsx
🔍 Processing: AR-Air 26.xlsx
🔍 Processing: AR-Air 27.xlsx
🔍 Processing: AR-Air 28.xlsx
🔍 Processing: AR-Air 29.xlsx
🔍 Processing: AR-Air 3.xlsx
🔍 Processing: AR-Air 30.xlsx
🔍 Processing: AR-Air 4.xlsx
🔍 Processing: AR-Air 5.xlsx
🔍 Processing: AR-Air 6.xlsx
🔍 Processing: AR-Air 7.xlsx
🔍 Processing: AR-Air 8.xlsx
🔍 Processing: AR-Air 9.xlsx
🔍 Processing: AR-Tin 1.xlsx
🔍 Processing: AR-Tin 10.xlsx
🔍 Processing: AR-Tin 11.xlsx
🔍 Processing: AR-Tin 12.xlsx
🔍 Processing: AR-Tin 13.